# Load the data

In [130]:
# Import necessary packages
using DrWatson
using JLD2

# Activate the project environment
@quickactivate :FlowComplexity

# Define the file path
file_path = joinpath("./data", "data.jld2")

# Load the data from the specified file
@load file_path sim_array

1-element Vector{Symbol}:
 :sim_array

# Show time series for all chemostats of simulation no. X

In [131]:
using StatsBase

function get_timeseries(sim)
    # sim = sim_array[1]
    # get the number of chemostats
    nchem = length(sim.time_evolution)
    # get the time steps
    t_arr = collect(keys(sim.time_evolution[1][:complete_timeseries]))
    nt = length(t_arr)
    # get the maximum variable
    vals = Iterators.flatten(values(sim.time_evolution[1][:complete_timeseries]))
    # nv = length(vals)
    maxvals = maximum(vals)

    # populate an array with the values
    ts = zeros(Int, nchem, nt, maxvals)
    for i in 1:nchem
        for j in 1:nt
            tt = t_arr[j]
            cm = countmap(sim.time_evolution[i][:complete_timeseries][tt])
            v = collect(keys(cm))
            nv = length(cm)
            for k in 1:nv
                kk = v[k]
                ts[i, j, kk] = cm[kk]
            end
        end
    end
    return t_arr, ts
end

t_arr, ts = get_timeseries(sim_array[1])

# display(t_arr)
display(ts) # chem t var


9×1001×137188 Array{Int64, 3}:
[:, :, 1] =
 1001  1001  1000  1000  1000  1000  …  1000  1000  1000  1001  1001  1000
    4     4     4     4     4     4        4     4     0     4     4     4
    0     0     0     0     0     0        0     0     0     0     0     0
    0     0     0     0     0     0        0     0     0     0     0     0
    0     0     0     0     0     0        0     0     0     0     0     0
    0     0     0     0     0     0  …     0     0     0     0     0     0
    0     0     0     0     0     0        0     0     0     0     0     0
    0     0     0     0     0     0        0     0     0     0     0     0
    0     0     0     0     0     0        0     0     0     0     0     0

[:, :, 2] =
 184  174  166  157  188  195  184  207  …  171  203  176  180  189  165  206
   0    0    0    0    0    0    0    0       0    0    0    0    0    0    0
   0    0    0    0    0    0    0    0       0    0    0    0    0    0    0
   0    0    0    0    0    0    0 

# old version

In [ ]:
using DataFrames, Plots, FileIO

sim_no = 2
sim = sim_array[sim_no]
nchem = 9

# Define the array that will hold our population time series
max_t = Int(sim.total_time) + 1 # our sim goes to 0...100 so index has to go 1...101
nspecies = 10
pop_array = zeros(Int, nchem, max_t, nspecies)

# Convert the time series to a DataFrame
time_series_df = FlowComplexity.convert_timeseries_to_tidy_df(sim.time_evolution)

# Populate the population array using efficient DataFrame operations
for row in eachrow(time_series_df)
    
    i = Int(row[:reactor])
    j = round(Int, row[:time]) + 1 # indexing starts at 1
    k = parse(Int, row[:variable]) # to account for absent data (i.e., zero)
    if k > nspecies
        continue
    end
    # println(row)
    # println("i=$i j=$j k=$k")
    pop_array[i, j, k] = row[:value]
end

# Create a 3x3 grid plot
plot_grid = plot(layout = (3, 3))

# Loop over each subplot
for i in 1:nchem
    # Extract the time series data for the i-th plot
    time_series_data = pop_array[i, :, :]
    
    # Create a subplot for each species in the time series
    p = plot()
    for j in 1:nspecies
        plot!(plot_grid[i], 1:max_t, time_series_data[:, j], label = "Species $j", legend = :none)
    end
end

display(plot_grid)

# new version

In [125]:
sim_array[94].time_evolution[1][:complete_timeseries]

Dict{Any, Any} with 1001 entries:
  719.0 => [1, 1, 2499, 3, 2, 1, 1758, 1, 8987, 1  …  1, 1, 7159, 763, 307, 713…
  858.0 => [323, 1, 1534, 579, 11898, 1, 3, 1, 277, 30093  …  9233, 150, 1, 326…
  699.0 => [10, 1676, 129, 15, 2214, 1, 1955, 1, 4032, 868  …  1, 22, 27, 3749,…
  831.0 => [10191, 16, 259, 1, 83, 12529, 13143, 2, 11, 1  …  82, 46, 1, 264, 3…
  983.0 => [3, 2, 1, 1, 1, 199, 1, 28, 1, 53  …  1, 1, 2, 5, 762, 1, 192, 1, 94…
  673.0 => [1, 6452, 3, 16, 21, 2, 1, 94, 37, 2409  …  227, 1522, 59, 71, 481, …
  319.0 => [1, 1, 3, 367, 1, 141, 1, 1, 1759, 1  …  1, 1, 698, 1, 3, 584, 2, 1,…
  73.0  => [279, 1, 577, 1, 669, 2, 123, 1, 8, 1  …  512, 185, 358, 490, 28, 64…
  251.0 => [1, 1, 2, 1, 1, 171, 1, 2, 469, 1241  …  1792, 1, 1425, 1, 98, 1, 1,…
  687.0 => [267, 1, 1, 334, 2, 3, 896, 701, 135, 2766  …  240, 1, 1, 114, 374, …
  115.0 => [1, 70, 53, 48, 873, 1, 25, 1, 2, 1  …  1, 23, 1, 1, 999, 43, 10, 1,…
  112.0 => [1, 309, 1, 1052, 4, 94, 2, 1, 2, 268  …  1, 1166, 1, 66, 1, 1, 

In [132]:
using DataFrames, Plots, FileIO

sim_no = 80
sim = sim_array[sim_no]
nchem = 9

# Define the array that will hold our population time series
max_t = Int(sim.total_time) + 1 # our sim goes to 0...100 so index has to go 1...101
nspecies = 10
pop_array = zeros(Int, nchem, max_t, nspecies)

# Convert the time series to a DataFrame
# time_series_df = FlowComplexity.convert_timeseries_to_tidy_df(sim.time_evolution)
t_arr, ts = get_timeseries(sim)
# ts[chem, t, var]

# # Populate the population array using efficient DataFrame operations
# for row in eachrow(time_series_df)
    
#     i = Int(row[:reactor])
#     j = round(Int, row[:time]) + 1 # indexing starts at 1
#     k = parse(Int, row[:variable]) # to account for absent data (i.e., zero)
#     if k > nspecies
#         continue
#     end
#     # println(row)
#     # println("i=$i j=$j k=$k")
#     pop_array[i, j, k] = row[:value]
# end

# Create a 3x3 grid plot
plot_grid = plot(size=(800,800), layout = (3, 3))

# Loop over each subplot
for i in 1:nchem
    # Extract the time series data for the i-th plot
    # time_series_data = pop_array[i, :, :]
    
    # Create a subplot for each species in the time series
    p = plot()
    for j in 1:nspecies
        y = ts[i,:,j]
        plot!(plot_grid[i], 1:max_t, y, ylim=(0, 1000), label = "Species $j", legend = :none)
    end
end

display(plot_grid)

BoundsError: BoundsError: attempt to access 9×1001×14686 Array{Int64, 3} at index [2, 107, 17765]

In [81]:
ts[2,2,2]

0

# Plot the Std(M) vs $k_d$ 

## (i.e., the std of monomers vs the outflow)

In [5]:
using DataFrames, Plots, FileIO, Statistics

first_sim = sim_array[1]
nsims = 100
nchem = 9
nspecies = 10
t = 100

# Define the array that will hold our data
pops = zeros(Float64, nsims, nchem, nspecies)

# Loop over all the simulations
for i in 1:100
    sim = sim_array[i]
    df = FlowComplexity.convert_timeseries_to_tidy_df(sim.time_evolution)
    df = filter(row -> row[:time] == t, df)

    for j in 1:nchem
        df_reactor = filter(row -> row[:reactor] == j, df)

        for k in 1:nspecies
            df_reactor_species = filter(row -> row[:variable] == string(k), df_reactor)
            if nrow(df_reactor_species) > 0
                pops[i, j, k] = df_reactor_species[1, :value]      
            end
        end
    end
end

In [ ]:
display(pops[2,:,:])

In [ ]:
stds = zeros(Float64, nsims, nspecies)

for i in 1:100
    for j in 1:nspecies
        stds[i, j] = std(pops[i, :, j])
        # stds = dropdims(stds, dims=2)
    end
end

display(stds)

In [ ]:
p = plot()

for i in 1:1
    plot!(p, 1:100, stds[:, i], legend= :none)
end

display(p)

In [ ]:
display(stds)

In [ ]:
std([1000,10,20,50,1,0,0,0,0])

In [ ]:
mu = zeros(Float64, nsims, nspecies)

for i in 1:100
    for j in 1:nspecies
        mu[i, j] = mean(pops[i, :, j])
        # stds = dropdims(stds, dims=2)
    end
end

display(mu)

In [ ]:
p = plot()

for i in 1:1
    plot!(p, 1:100, mu[:, i], legend= :none)
end

display(p)